In [ ]:
!pip install imbalanced-learn
!pip install vecstack
!pip install torch torchvision


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached torch-2.5.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached torch-2.4.1-cp312-cp312-win_amd64.whl.metadata (27 kB)
Using cached torch-2.4.1-cp312-cp312-win_amd64.whl (199.4 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.4.1+cu124 requires torch==2.4.1+cu124, but you have torch 2.4.1 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
# Import necessary libraries
from vecstack import stacking
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [15]:

train_data = pd.read_csv('RevisedHomesiteTrain1.csv')
test_data = pd.read_csv('RevisedHomesiteTest1.csv')

In [16]:
# Preprocessing
X_train = train_data.drop(['QuoteConversion_Flag'], axis=1)  # Replace 'target' with the name of the target column
y_train = train_data['QuoteConversion_Flag']

# one-hot encoded - Convert categorical variables into dummy/indicator variables
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(test_data)

# Align columns in X_train and X_test to ensure consistency
X_train, X_test = X_train.align(X_test, join='inner', axis=1)

# Print the shape of encoded data
print("Shape of training data after one-hot encoding:", X_train.shape)
print("Shape of test data after one-hot encoding:", X_test.shape)

Shape of training data after one-hot encoding: (65000, 599)
Shape of test data after one-hot encoding: (173836, 599)


In [17]:
# Define the models
mlp = MLPClassifier(random_state=42)
svc = LinearSVC(random_state=42)
dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)
knn = KNeighborsClassifier()

In [18]:
# Train the models individually
models = [mlp, dt, rf, knn, svc]
model_names = ['MLP', 'Decision Tree', 'Random Forest', 'KNN','Linear SVM']

for model, name in zip(models, model_names):
    # Fit the model and Predict
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    prediction_df = pd.DataFrame({'QuoteNumber': X_test['QuoteNumber'], 'QuoteConversion_Flag': y_pred})
    file_name = f'prediction_Data_{name}.csv'
    prediction_df.to_csv(file_name, index=False)
    print(f"Predictions saved to {file_name}")

Predictions saved to prediction_Data_MLP.csv
Predictions saved to prediction_Data_Decision Tree.csv
Predictions saved to prediction_Data_Random Forest.csv
Predictions saved to prediction_Data_KNN.csv
Predictions saved to prediction_Data_Linear SVM.csv


In [19]:
# Print class distribution before SMOTE
print("Before SMOTE:")
print("Class distribution in training set:", y_train.value_counts())

# Apply SMOTE for oversampling the minority class
smote = SMOTE(sampling_strategy=0.6, random_state=42)  # Adjust percentage as needed
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Print class distribution after SMOTE
print("After SMOTE:")
print("Class distribution in training set:", y_train_smote.value_counts())

Before SMOTE:
Class distribution in training set: QuoteConversion_Flag
0    52738
1    12262
Name: count, dtype: int64
After SMOTE:
Class distribution in training set: QuoteConversion_Flag
0    52738
1    31642
Name: count, dtype: int64


In [20]:
S_train, S_test = stacking(models=[mlp, svc, dt, rf, knn],
    X_train=X_train_smote,
    y_train=y_train_smote,
    X_test=X_test,
    regression=False,
    mode='oof_pred_bag',
    needs_proba=False,
    save_dir=None,
    metric=accuracy_score,
    n_folds=4,
    stratified=True,
    shuffle=True,
    random_state=42,
    verbose=2
)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [MLPClassifier]
    fold  0:  [0.88248400]
    fold  1:  [0.77435411]
    fold  2:  [0.79530694]
    fold  3:  [0.86082010]
    ----
    MEAN:     [0.82824129] + [0.04469941]
    FULL:     [0.82824129]

model  1:     [LinearSVC]
    fold  0:  [0.88656080]
    fold  1:  [0.89224935]
    fold  2:  [0.90727661]
    fold  3:  [0.90926760]
    ----
    MEAN:     [0.89883859] + [0.00967118]
    FULL:     [0.89883859]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.90286798]
    fold  1:  [0.90590187]
    fold  2:  [0.90334202]
    fold  3:  [0.90040294]
    ----
    MEAN:     [0.90312870] + [0.00195152]
    FULL:     [0.90312870]

model  3:     [RandomForestClassifier]
    fold  0:  [0.92562219]
    fold  1:  [0.92543257]
    fold  2:  [0.92789761]
    fold  3:  [0.92472150]
    ----
    MEAN:     [0.92591846] + [0.00119097]
    FULL:     [0.9

In [ ]:
#STACKING - CONTRUCT A GRADIENT BOOSTING MODEL
meta_model = GradientBoostingClassifier(random_state=42)
stacking_clf = meta_model.fit(S_train, y_train_smote)
y_pred_val_stack  = stacking_clf.predict(S_test)

In [ ]:
prediction_df = pd.DataFrame({'QuoteNumber': X_test['QuoteNumber'], 'QuoteConversion_Flag': y_pred_val_stack})

# Store y_test_pred to data file
file_name = f'prediction_Data.csv'
prediction_df.to_csv(file_name, index=False)
print(f"Predictions saved to {file_name}")

Predictions saved to prediction_Data.csv


In [ ]:
# hyperparameter tuning - Define the parameter grid for the meta-model
param_grid = {
    'n_estimators': [50, 100],
    'learning_rate': [0.1, 0.2],
    'max_depth': [3, 5],
    'min_samples_split': [2, 5]
}

# Use GridSearchCV to find the best hyperparameters for the meta-model
grid_search = GridSearchCV(stacking_clf, param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(S_train, y_train_smote)

# Get the best meta-model
best_meta_model = grid_search.best_estimator_
print("Best parameters for the stacked model:", best_meta_model)

Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best parameters for the stacked model: GradientBoostingClassifier(learning_rate=0.01, random_state=42)


In [ ]:
# Predict and evaluate the stacked model
y_pred_val_stack_tuned = best_meta_model.predict(S_test)

prediction_df = pd.DataFrame({'QuoteNumber': X_test['QuoteNumber'], 'QuoteConversion_Flag': y_pred_val_stack_tuned})

# Store y_test_pred to data file
file_name = f'prediction_Data_tuned.csv'
prediction_df.to_csv(file_name, index=False)
print(f"Predictions saved to {file_name}")

Predictions saved to prediction_Data_tuned.csv
